<h1 align = 'center'>XGBoost<h1>
<h1 align = 'center'>Đỗ Trọng Khánh - 19521676<h1>

#Connect Driver

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import Libary

In [19]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
scaler = StandardScaler()
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, shuffle=True, random_state=1)

## Bài 1: Sử dụng Gradient Boosting Machine để thực hiện trên dữ liệu Data.csv

## Đọc dữ liệu từ drive

In [20]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Lập trình Python cho Máy học/Week 11/Data.csv')
data.head()

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [21]:
X_data = data.drop('Class', 1)
y_data = data['Class']

X_data = X_data.to_numpy()
y_data = y_data.to_numpy()

## Sử dụng KFold đánh giá bộ dữ liệu

In [22]:
acc_list = []
for train, test in kf.split(X_data):
    X_train, X_test = X_data[train], X_data[test]
    y_train, y_test = y_data[train], y_data[test]

    X_train_scaler = scaler.fit_transform(X_train)
    X_test_scaler = scaler.fit_transform(X_test)

    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train_scaler, y_train)
    y_pred = clf.predict(X_test_scaler)
    acc_list.append(accuracy_score(y_pred, y_test))

print("Accuracy: {}%".format(np.max(acc_list) * 100))

Accuracy: 98.54014598540147%


# Bài 2: XGBoost

## Sử dụng KFold đánh giá bộ dữ liệu

In [23]:
acc_list = []
for train, test in kf.split(X_data):
    X_train, X_test = X_data[train], X_data[test]
    y_train, y_test = y_data[train], y_data[test]

    X_train_scaler = scaler.fit_transform(X_train)
    X_test_scaler = scaler.fit_transform(X_test)

    xgb_model = XGBClassifier()
    xgb_model.fit(X_train_scaler, y_train)
    y_pred = xgb_model.predict(X_test_scaler)

    acc_list.append(accuracy_score(y_pred, y_test))

print("Accuracy: {}%".format(np.max(acc_list) * 100))

Accuracy: 99.27007299270073%


# Bài 3: Làm bài tập 1 và bài tập 2 với dataset loan_data.csv

## Đọc dữ liệu từ drive

In [24]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Lập trình Python cho Máy học/Week 11/loan_data.csv')
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [25]:
data = data.dropna() #Xóa các hàng có chứa NaN
data = data.drop('Loan_ID', 1) #Loại bỏ cột Loan_ID yếu tố không ảnh hưởng đến cho vay

## Sử dụng One Hot Encoder và Label Binarizer

In [26]:
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer
one_hot = OneHotEncoder()
label_binarizer = LabelBinarizer()

In [27]:
gender_col = data['Gender'].to_numpy()
married_col = data['Married'].to_numpy()
education_col = data['Education'].to_numpy()
self_employed_col = data['Self_Employed'].to_numpy()

data['Gender'] = label_binarizer.fit_transform(gender_col)
data['Married'] = label_binarizer.fit_transform(married_col)
data['Education'] = label_binarizer.fit_transform(education_col)
data['Self_Employed'] = label_binarizer.fit_transform(self_employed_col)

In [28]:
property_area_col = data['Property_Area']
dependents_col = data['Dependents']
data = data.drop(data.columns[[2, 10]], axis = 1)

property_area_col = property_area_col.to_numpy().reshape(-1, 1)
property_area_col = one_hot.fit_transform(property_area_col).toarray()

dependents_col = dependents_col.to_numpy().reshape(-1, 1)
dependents_col = one_hot.fit_transform(dependents_col).toarray()

In [29]:
y_data = data['Loan_Status'].to_numpy()
y_data = label_binarizer.fit_transform(y_data).ravel()

X_data = data.drop('Loan_Status', axis = 1).to_numpy()

In [30]:
X_data = np.hstack((X_data, property_area_col))
X_data = np.hstack((X_data, dependents_col))

## Sử dụng KFold đánh giá bộ dữ liệu

### Gradient Boosting Machine

In [31]:
acc_list = []
for train, test in kf.split(X_data):
    X_train, X_test = X_data[train], X_data[test]
    y_train, y_test = y_data[train], y_data[test]

    X_train_scaler = scaler.fit_transform(X_train)
    X_test_scaler = scaler.fit_transform(X_test)

    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train_scaler, y_train)
    y_pred = clf.predict(X_test_scaler)
    acc_list.append(accuracy_score(y_pred, y_test))

print("Accuracy: {}%".format(np.max(acc_list) * 100))

Accuracy: 81.25%


### XGBoost

In [32]:
acc_list = []
for train, test in kf.split(X_data):
    X_train, X_test = X_data[train], X_data[test]
    y_train, y_test = y_data[train], y_data[test]

    X_train_scaler = scaler.fit_transform(X_train)
    X_test_scaler = scaler.fit_transform(X_test)

    xgb_model = XGBClassifier()
    xgb_model.fit(X_train_scaler, y_train)
    y_pred = xgb_model.predict(X_test_scaler)

    acc_list.append(accuracy_score(y_pred, y_test))

print("Accuracy: {}%".format(np.max(acc_list) * 100))

Accuracy: 83.33333333333334%
